In [19]:
import pandas as pd
from sklearn import tree

In [20]:
df = pd.read_csv("/Users/jlh/global-health-map/data/separated_gender.csv")
df = df.drop(columns=['Unnamed: 0'])
df.head()

,Location,Period,Gender,Life_Expectancy,Healthy_Life_Expectancy,%Mrt30_70_crdo_cncr_dbts_rsprtry,Suicide100000,Ttl_alc_percapita_15+,Posioning100000,Smoking15+
0,Albania,2015,Male,76.10,67.99,20.9,7.6,10.92,0.48,51.4
1,Albania,2015,Female,79.69,70.12,13.7,4.4,2.53,0.37,8.3
2,Albania,2010,Male,74.23,66.34,22.3,9.5,12.47,0.59,53.2
3,Albania,2010,Female,78.31,68.85,14.6,6.1,2.98,0.44,9.1
4,Albania,2000,Male,70.65,63.15,23.1,7.4,10.60,1.29,57.4


In [21]:
X = df.drop(columns=['Location', 'Period', 'Gender'])
y = df["Gender"]
print(X.shape, y.shape)

(860, 7) (860,)


In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [23]:
from sklearn.preprocessing import LabelEncoder

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
label_encoder.fit(y_test)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [24]:
from keras.utils import to_categorical

# Step 2: One-hot encoding
hot_y_train = to_categorical(encoded_y_train)
hot_y_test = to_categorical(encoded_y_test)
hot_y_train

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [25]:
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [31]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, hot_y_train)
clf.score(X_test, hot_y_test)

0.9209302325581395

In [32]:
feature_names = X.columns

In [33]:
sorted(zip(clf.feature_importances_, feature_names), reverse=True)

[(0.4730898935737658, 'Ttl_alc_percapita_15+'),
 (0.22729879116719318, 'Smoking15+'),
 (0.18077598770449757, 'Life_Expectancy'),
 (0.047926564013148834, 'Posioning100000'),
 (0.03181171365803696, 'Suicide100000'),
 (0.02253668763102725, '%Mrt30_70_crdo_cncr_dbts_rsprtry'),
 (0.0165603622523304, 'Healthy_Life_Expectancy')]

In [34]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

In [35]:
DecisionTreeClassifier.get_params(DecisionTreeClassifier)

/Users/jlh/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


{'ccp_alpha': None,
 'class_weight': None,
 'criterion': None,
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': None,
 'min_impurity_split': None,
 'min_samples_leaf': None,
 'min_samples_split': None,
 'min_weight_fraction_leaf': None,
 'presort': None,
 'random_state': None,
 'splitter': None}

In [39]:
scoring = {'AUC': 'roc_auc'}
param_grid = {'criterion':['gini', 'entropy'], 'max_depth':[5, 7, 10, 15, 25, 40, 65, 95, 150]}

gs = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5)
gs.fit(X_train, hot_y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [5, 7, 10, 15, 25, 40, 65, 95, 150]})

In [40]:
print(gs.best_params_)
print(gs.best_score_)
#https://stackoverflow.com/questions/38709690/scikit-learn-using-gridsearchcv-on-decisiontreeclassifier

{'criterion': 'entropy', 'max_depth': 40}
0.9209302325581395


In [44]:
predictions = gs.predict(X_test_scaled)
gs.score(X_test, hot_y_test)

0.9209302325581395